In [22]:
import sys
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime
import tensorflow as tf
import tensorflow.keras as keras

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Note
Invalid - accidentally averaged the values of the past

# Loading data 

In [23]:
from with_linear_acc import Session
sessions = Session.load_multiple_session('./data')
frames, df = Session.concatenate_multiple_sessions(sessions)

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5d130378b280] moov atom not found


KeyboardInterrupt: 

In [ ]:
df = sessions[0].data['angular_speed_control_df']

In [ ]:
asd0 = df['angular_velocity'].rolling(100).apply(mean_nominmax)#.shift(99)

asd = df['angular_velocity'].rolling(50).apply(mean_nominmax)#.shift(49)
asd1 = df['angular_velocity'].rolling(3).apply(mean_nominmax)#.shift(2)


px.line(y=[asd1, asd, asd0])

# Select data for the model

In [ ]:
def prepare_to_tensorflow(frames, df):

    # data selection
    df = df.query('speed!=0')

    # match the data with the frame
    frame2data = df.groupby('exact_total_frame_idx')[['linacc0', 'linacc1', 'linacc2']].mean()
    outputs = df.groupby('offset_total_frame_idx')[['angular_velocity_smooth']].mean()

    # the reindexed data
    frame_indices = outputs.index.values

    frames_reindexed = frames[frame_indices]
    other_inputs = frame2data.reindex(frame_indices)

    # screen out the nans
    nafilter = (~other_inputs.isna().any(axis=1)).values

    frames_reindexed = frames_reindexed[nafilter]
    other_inputs = other_inputs.values[nafilter]
    outputs = outputs.values[nafilter]
    
    
    return frames_reindexed, other_inputs/10, outputs/180

In [ ]:
frames_in, other_in, out = prepare_to_tensorflow(frames, df)

In [ ]:
from train_tools import validation_chunk_split

In [ ]:
train_idx, val_idx = validation_chunk_split(len(frames_in), val_split=0.2)
train_frames, train_other, train_out = frames_in[train_idx], other_in[train_idx], out[train_idx]
val_frames, val_other, val_out = frames_in[val_idx], other_in[val_idx], out[val_idx]

# Model

In [24]:
def get_model(lr=.1, other_metrics=[]):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3

    img_aug_preprocess_layers = [
        # keras.layers.RandomTranslation(0.05, 0.05, fill_mode='reflect'),
        # keras.layers.RandomRotation(0.02, fill_mode='reflect'),
        # keras.layers.RandomZoom(0.05, fill_mode='reflect'),
        # keras.layers.RandomContrast(0.5),
        # keras.layers.RandomBrightness([-0.7, 0.7])
    ]
    
    cnn_layers =  [
        keras.layers.Conv2D(16, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(32, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(64, 5, strides=2, activation='relu'),         
        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.Conv2D(96, 3, activation='relu'),         
        keras.layers.Flatten(),
        keras.layers.Dropout(0.2),
    ]
    
    cnn = keras.Sequential(
        [
            keras.layers.InputLayer(image_shape),
            keras.layers.Rescaling(1/255), 
            *img_aug_preprocess_layers,
            *cnn_layers
        ])

    other_inputs = keras.layers.InputLayer((3,))
    x = keras.layers.Concatenate()([cnn.outputs[0], other_inputs.output])
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    y = keras.layers.Dense(1)(x)

    model = keras.Model([cnn.inputs[0], other_inputs.output], y)


    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE']+other_metrics)

    return model 

model = get_model()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 114,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 64, 114,   │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 30, 55,    │      1,216 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 13, 26,    │     12,832 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 5, 11, 64) │     51,264 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 3, 9, 64)  │     36,928 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 1, 7, 96)  │     55,392 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 672)       │          0 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 672)       │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 675)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     86,528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 252,481 (986.25 KB)

 Trainable params: 252,481 (986.25 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
from train_tools import find_lr, tg_notify, use_tensorboard, end_epoch_notify

In [26]:
find_lr(get_model(), [train_frames, train_other], train_out)
#tensorboard --logdir src/training/logs --bind_all

Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - MAE: 0.1644 - loss: 0.0269 - learning_rate: 1.0000e-06
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1634 - loss: 0.0266 - learning_rate: 1.1220e-06
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MAE: 0.1621 - loss: 0.0261 - learning_rate: 1.2589e-06
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MAE: 0.1621 - loss: 0.0261 - learning_rate: 1.4125e-06
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1637 - loss: 0.0262 - learning_rate: 1.5849e-06
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MAE: 0.1637 - loss: 0.0267 - learning_rate: 1.7783e-06
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MAE: 0.1624 - loss: 0.0260 - learning_rate: 1.9953e-06
Epoch 8/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1610 - loss: 0.0257 - learning_rate: 2.2387e-06
Epoch 9/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - MAE: 0.1602 - loss: 0.0253 - learning_rate: 2.5119e-06
Epoch 10/

In [27]:
model = get_model(1e-4)
model.fit(
    [train_frames, train_other],
    train_out,
    epochs=100, 
    validation_data = ([val_frames, val_other], val_out),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
        use_tensorboard('training'), 
        end_epoch_notify()
        ] 
    )

Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - MAE: 0.1629 - loss: 0.0261 - val_MAE: 0.1642 - val_loss: 0.0266
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1538 - loss: 0.0228 - val_MAE: 0.1554 - val_loss: 0.0234
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1432 - loss: 0.0197 - val_MAE: 0.1465 - val_loss: 0.0202
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1299 - loss: 0.0164 - val_MAE: 0.1502 - val_loss: 0.0218
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1207 - loss: 0.0145 - val_MAE: 0.1285 - val_loss: 0.0162
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.1040 - loss: 0.0109 - val_MAE: 0.1141 - val_loss: 0.0130
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.0962 - loss: 0.0095 - val_MAE: 0.1062 - val_loss: 0.0119
Epoch 8/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - MAE: 0.0862 - loss: 0.0075 - val_MAE: 0.1005 - val_loss: 0.0107
Epoch 9/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms

In [ ]:
#frames_reindexed#, other_inputs_reindexed, outputs

In [31]:

#train_frames, train_other, train_out

y_pred = model.predict([train_frames[:], train_other[:]], batch_size=64)

176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [34]:
px.line(np.c_[train_out[:idx, 0], y_pred[:idx, 0]])

In [36]:
y_pred = model.predict([val_frames[:], val_other[:]], batch_size=64)

px.line(np.c_[val_out[:idx, 0], y_pred[:idx, 0]])

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [ ]:
model.save('10Jun-pi.keras')